In [ ]:
import pandas as pd
import numpy as np

# IO

In [ ]:
from PineBioML.preprocessing import IO


In [ ]:
# Path
input_folder = "./input/"
output_folder = "./output/"
export_title = "example_basic "

In [ ]:
##### Read data method 1: read single file
data = IO.read_file(input_folder+ "example_data.xlsx", index_col=0)
y = data["y"]
x = data.drop("y", axis = 1)


# Preprocessing

In [ ]:
### within sample normalization
#x = x.T
#x = (x-x.mean())/x.std()
#x = x.T

### standarization
x = (x-x.mean())/x.std()

In [ ]:
# Impute 0
from PineBioML.preprocessing import impute

# The imputer will drop the features with missing value more than 50%.
# Remaining will be filled by 0.
simple = impute.simple_imputer(threshold = 0.6, strategy = "constant")
x, y = simple.fit_transform(x, y)

# Analysis utils

In [ ]:
import PineBioML.report.utils as report_utils

In [ ]:
report_utils.data_overview(x, y)

# Feature selection

In [ ]:
from PineBioML.selection.ensemble import selector

In [ ]:
# Use several way to select features, which includes decision tree, random forest, lasso, multi lasso, svm, adaboost, xgboost
ensemble_selector = selector()
important_feature, importance = ensemble_selector.Select(x, y, 10)

In [ ]:
# The features that was selected by ML algoritms.
important_feature

In [ ]:
# The feature importance given by diffirent methods.
# Please Notice that the score between diffirent methods is not comparable.
importance

In [ ]:
merge_importance = ((importance-importance.mean())/importance.std()).mean(axis = 1).sort_values(ascending=False)
merge_importance

In [ ]:
report_utils.data_overview(x[merge_importance.index[:5]], y)

# Modeling

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x[merge_importance.index[:5]], y, test_size=0.33, random_state=142, shuffle=True)

In [ ]:
import PineBioML.model.supervised.Classification as tuner

### optuna

In [ ]:
rf = tuner.RandomForest_tuner(x_train, y_train, target="f1").tune()

In [ ]:
svm = tuner.SVC_tuner(x_train, y_train, target="roc_auc").tune()

In [ ]:
# fit the model
rf.fit(x_train, y_train)
svm.fit(x_train, y_train)

In [ ]:
report_utils.classification_summary(y_valid ,rf.predict_proba(x_valid)[:, 1]) 

In [ ]:
report_utils.classification_summary(y_valid, svm.predict_proba(x_valid)[:, 1]) 

### Mljar

In [ ]:
from supervised.automl import AutoML

mljar = AutoML(mode = "Perform", results_path = output_folder+"MLjar")
mljar.fit(x_train, y_train)

In [ ]:
report_utils.classification_summary(y_valid, mljar.predict_proba(x_valid)[:, 1])

# Output

In [ ]:
# Save the result ro output_path
important_feature.to_csv(output_folder + export_title + " important feature.csv")
importance.to_csv(output_folder+export_title+ " importance.csv")

In [ ]:
import joblib
# save model
joblib.dump(rf, output_folder+"models/"+export_title+"RandomForest")
joblib.dump(svm, output_folder+"models/"+export_title+"SVM")

In [ ]:
# load model
presaved_model = joblib.load(output_folder+"models/"+export_title+"SVM")
report_utils.classification_summary(y_valid, presaved_model.predict_proba(x_valid)[:, 1]) 